In [77]:
import json

with open('data/rpfs_pos/USpowerGrid.json') as f:
    jsondata = json.load(f)


In [78]:
import numpy as np


tq = 'stress'

data = []
indices = []


for i,e in enumerate(jsondata['e']):
    params = e['params']
    qs = []
    n_len = None
    for seed in e['seed']:
        t = e['seed'][seed]
        qs.append(t[tq])
        n_len = len([p for p in t['pos']])

    data.append([np.mean(np.array(qs)), params['edge_length'], params['number_of_pivots'], params['number_of_iterations'], params['eps']])
    # data.append([np.mean(np.array(qs)), (params['edge_length'] - 1) / (100 - 1), (params['number_of_pivots'] - 1) / (n_len - 1), (params['number_of_iterations'] - 1) / (100 - 1), (params['eps'] - 0.01)/(1 - 0.01)])
    indices.append(i)


In [79]:
import pandas as pd

df = pd.DataFrame(data, columns=[
                  tq, 'edge_length', 'number_of_pivots', 'number_of_iterations', 'eps'])


In [80]:
x = df[['edge_length', 'number_of_pivots', 'number_of_iterations', 'eps']]
y = df[[tq]]


In [81]:
xss_pd = (x - x.mean()) / x.std()
yss_pd = (y - y.mean()) / y.std()
# xss_pd = x
# yss_pd = y


In [82]:
x['number_of_iterations'].min()

60

In [83]:
from sklearn.linear_model import LinearRegression

model_lr_std = LinearRegression()
model_lr_std.fit(xss_pd, yss_pd)

print(list(x.columns))
print(model_lr_std.coef_[0])
print(model_lr_std.intercept_)
print(model_lr_std.score(xss_pd, yss_pd))

['edge_length', 'number_of_pivots', 'number_of_iterations', 'eps']
[ 0.96479778  0.15040917 -0.05715426 -0.03797496]
[-1.4768147e-16]
0.9200841880947177


In [84]:
import statsmodels.api as sm

x_add_const = sm.add_constant(xss_pd)
model_sm = sm.OLS(yss_pd, x_add_const).fit()
print(model_sm.summary())

                            OLS Regression Results                            
Dep. Variable:                 stress   R-squared:                       0.920
Model:                            OLS   Adj. R-squared:                  0.899
Method:                 Least Squares   F-statistic:                     43.17
Date:                Wed, 02 Nov 2022   Prob (F-statistic):           4.65e-08
Time:                        23:05:24   Log-Likelihood:                -2.5980
No. Observations:                  20   AIC:                             15.20
Df Residuals:                      15   BIC:                             20.17
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                         0 

In [85]:
xss_pd['edge_length']

0    -1.578027
1     1.261610
2    -0.888401
3     0.815381
4     0.085189
5     0.653117
6    -0.036510
7     1.545574
8    -0.726136
9    -0.158208
10   -1.618593
11   -1.010099
12   -0.807268
13    0.896514
14    0.774815
15    0.653117
16   -0.401606
17    0.571984
18   -1.334629
19    1.302176
Name: edge_length, dtype: float64